In [0]:
!pip install twint

     |████████████████████████████████| 1.2MB 16.2MB/s 
     |████████████████████████████████| 245kB 48.3MB/s 
     |████████████████████████████████| 92kB 12.3MB/s 
     |████████████████████████████████| 358kB 40.3MB/s 
     |████████████████████████████████| 256kB 41.0MB/s 
     |████████████████████████████████| 204kB 47.2MB/s 
  Created wheel for twint: filename=twint-2.1.7-cp36-none-any.whl size=32262 sha256=eddabe62fc6b0ac35dcfff68a8fe172342a7e6cb5b5260f5ca506b7b2d4fffad
  Stored in directory: /root/.cache/pip/wheels/4c/9d/68/8e99024df8ce40eb5138d6057e8088afe03350e416964cf1a7
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13485 sha256=6439d8ad0aab283cab2b23c7f95bbe1cc4be50813810d31c2095e18f1e738d0c
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3163 sha256=8ea9a135cc4d97117dc044c180d832fcfb3d38ef5ccac

In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
import twint
import re
from bs4 import BeautifulSoup
import pandas as pd
import lxml
from nltk.tokenize import WordPunctTokenizer
from textblob import TextBlob
import io
import os

In [0]:
def load_tweet(keyword, limit):

  result_loc = 'tweets.csv'
  c = twint.Config()
  c.Search = keyword
  c.Limit = limit
  c.Lang = 'id'
  c.Custom["tweet"] = ['date','tweet','username']
  c.Output = result_loc
  c.Store_csv = True

  # asyncio.set_event_loop(asyncio.new_event_loop())
  twint.run.Search(c)

  df = pd.read_csv(result_loc)
  os.remove(result_loc)

  return df

In [0]:
def tweet_cleaner(text):
  tok = WordPunctTokenizer()
  pat1 = r'@[A-Za-z0-9]+'
  pat2 = r'https?://[A-Za-z0-9./]+'
  pat3 = r'pic.twitter.com/[A-Za-z0-9./]+'
  combined_pat = r'|'.join((pat1, pat2, pat3))
  soup = BeautifulSoup(text, 'lxml')
  souped = soup.get_text()
  stripped = re.sub(combined_pat, '', souped)
  try:
    clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
  except:
    clean = stripped
  letters_only = re.sub("[^a-zA-Z]", " ", clean)
  lower_case = letters_only.lower()
  words = tok.tokenize(lower_case)
  return (" ".join(words)).strip()

In [0]:
def word_normalize(text, norm_dic):
  norm = norm_dic

  norm_dic = pd.Series(norm.hasil.values,index=norm.singkat).to_dict()  
  text_tokenized = nltk.word_tokenize(text.lower())
  text = " ".join(word if word not in norm_dic else norm_dic[word] for word in text_tokenized)
  return text

In [0]:
def sentimenLabeling(text):
  analyticts  = TextBlob(text)
  an = analyticts
  
  try:
    if an.detect_language() != 'en':
      an = analyticts.translate(from_lang='id', to='en')
      print(an)
  except:
    print("Skip")
  
  if an.sentiment.polarity > 0:
    return 1
  elif an.sentiment.polarity == 0:
    return 0
  else:
    return -1

In [0]:
keyword = "HariGuruNasional2019"
limit = 500
tweet_data = None 
df = None

df = load_tweet(keyword,limit)

1198894954875179008 2019-11-25 09:23:26 UTC <KetawaYuk8> semangat amat abang2 nge-ayun-nyaa  #videongakak #ngakakkocak #HariGuruNasional2019 #GurukuSayang #ReuniBatal212Bubar #SOLVEHANBINsCASE #RecehkanTwitter pic.twitter.com/HqhKKn9Qmz
1198894945588793344 2019-11-25 09:23:24 UTC <DG9996DG> #HariGuruNasional2019 #HappyTeachersDay   Aku bingung mau mulai dari mana nyeritain ke-uwu-an Sam & murid-muridnya.  Okay.... Pertama-tama, Sam itu seorang guru tari—ballet lebih spesifiknya. Sam masih muda kok, 20 tahun aja belom nyampe.  Apa tidak keren? Apa tidak uwu?
1198894852819152896 2019-11-25 09:23:01 UTC <DPP_PKB> Selamat Hari Guru 2019 "Tahun ini harus menjadi titik tolak perbaikan manajemen guru di tanah air karena masih dalam perencanaan awal dan kita juga memiliki Menteri baru dg visi dan cara pandang yang masih segar," ujar @SyaifulHooda #HariGuruNasional2019  https://pkb.id/read/10580/refleksi-hari-guru-2019-syaiful-huda-perbaiki-manajemen-pendidik/ …
1198894711290900481 2019-11-25 0

In [0]:
df['tweet'] = df['tweet'].apply(lambda text: tweet_cleaner(text))
df

,date,tweet,username
0,2019-11-25,semangat amat abang nge ayun nyaa videongakak ...,ketawayuk8
1,2019-11-25,harigurunasional happyteachersday aku bingung ...,dg9996dg
2,2019-11-25,selamat hari guru tahun ini harus menjadi titi...,dpp_pkb
3,2019-11-25,selamat hari guru nasional kepada seluruh guru...,evolveml
4,2019-11-25,selamat hari guru harigurunasional harigurunas...,davfar17
...,...,...,...
495,2019-11-25,selamat hari guru harigurunasional,bebekcegukan
496,2019-11-25,harigurunasional selamat hari guru pendidikan ...,abd_chafidh
497,2019-11-25,selamat hari guru sungguh besar jasamu harigur...,olline1010
498,2019-11-25,harigurunasional maapkan muridmu ini pak bu ya...,kudapony41


In [0]:
norm = pd.read_csv('key_norm.csv')
df["tweet"] = df["tweet"].apply(lambda x: word_normalize(x, norm))
df

,date,tweet,username
0,2019-11-25,semangat amat abang nge ayun nyaa videongakak ...,ketawayuk8
1,2019-11-25,harigurunasional happyteachersday saya bingung...,dg9996dg
2,2019-11-25,selamat hari guru tahun ini harus menjadi titi...,dpp_pkb
3,2019-11-25,selamat hari guru nasional kepada seluruh guru...,evolveml
4,2019-11-25,selamat hari guru harigurunasional harigurunas...,davfar17
...,...,...,...
495,2019-11-25,selamat hari guru harigurunasional,bebekcegukan
496,2019-11-25,harigurunasional selamat hari guru pendidikan ...,abd_chafidh
497,2019-11-25,selamat hari guru sungguh besar jasamu harigur...,olline1010
498,2019-11-25,harigurunasional maapkan muridmu ini pak bu ya...,kudapony41


In [0]:
df["sentiment"] = df["tweet"].apply(lambda x: sentimenLabeling(x))
df

the spirit is very big brother is swinging the video video is not hilarious the day of the national badges that the reunibatal broke up solvehanbinscase dimekantwitter
harigurunasional happyteachersday I am confused about where to start to tell the students the students okay first of all sam is a ballet dance teacher more specifically sam still young how come the years have not been up to what is not cool or not uwu
happy teacher's day this year should be a starting point for improving teacher management in the country because it is still in the initial planning and we also have a new minister with a vision and perspective that is still fresh
happy national teacher's day to all teachers in Indonesia thank you for all your services that educate the national life of harigurunasional emlinstructors
happy teacher's day national day national day bad nationality teacher rich people who know how it goes reply
thank you my beloved success to you my children happy teacher's day harigurunasional

,date,tweet,username,sentiment
0,2019-11-25,semangat amat abang nge ayun nyaa videongakak ...,ketawayuk8,-1
1,2019-11-25,harigurunasional happyteachersday saya bingung...,dg9996dg,1
2,2019-11-25,selamat hari guru tahun ini harus menjadi titi...,dpp_pkb,1
3,2019-11-25,selamat hari guru nasional kepada seluruh guru...,evolveml,1
4,2019-11-25,selamat hari guru harigurunasional harigurunas...,davfar17,1
...,...,...,...,...
495,2019-11-25,selamat hari guru harigurunasional,bebekcegukan,0
496,2019-11-25,harigurunasional selamat hari guru pendidikan ...,abd_chafidh,0
497,2019-11-25,selamat hari guru sungguh besar jasamu harigur...,olline1010,0
498,2019-11-25,harigurunasional maapkan muridmu ini pak bu ya...,kudapony41,0


In [0]:
df["keyword"] = keyword
df

,date,tweet,username,sentiment,keyword
0,2019-11-25,semangat amat abang nge ayun nyaa videongakak ...,ketawayuk8,-1,HariGuruNasional2019
1,2019-11-25,harigurunasional happyteachersday saya bingung...,dg9996dg,1,HariGuruNasional2019
2,2019-11-25,selamat hari guru tahun ini harus menjadi titi...,dpp_pkb,1,HariGuruNasional2019
3,2019-11-25,selamat hari guru nasional kepada seluruh guru...,evolveml,1,HariGuruNasional2019
4,2019-11-25,selamat hari guru harigurunasional harigurunas...,davfar17,1,HariGuruNasional2019
...,...,...,...,...,...
495,2019-11-25,selamat hari guru harigurunasional,bebekcegukan,0,HariGuruNasional2019
496,2019-11-25,harigurunasional selamat hari guru pendidikan ...,abd_chafidh,0,HariGuruNasional2019
497,2019-11-25,selamat hari guru sungguh besar jasamu harigur...,olline1010,0,HariGuruNasional2019
498,2019-11-25,harigurunasional maapkan muridmu ini pak bu ya...,kudapony41,0,HariGuruNasional2019


In [0]:
df.to_csv('tweet.csv', encoding='utf-8')